# Generator using GTP3

Ferramenta que pega no conteudo das noticias (em formato json), faz sumarization para encontrar o main text para os posts.

O tweet tem os seguintes elementos: 
* Texto;
* Hashtags relevantes;
* Emojis;
* URL;


In [1]:
import os
import re
import json
import pandas as pd
import random
import nltk
nltk.download('stopwords')
nltk.download('omw-1.4')
from keybert import KeyBERT
import emoji #https://carpedm20.github.io/emoji/

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/mariabarbosa/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/mariabarbosa/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
/opt/homebrew/anaconda3/envs/py3/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Gerador

In [2]:
'''
Import open ai key
'''

import os
import openai

from dotenv import load_dotenv

load_dotenv()

OPENAI_KEY= os.getenv('OPENAI_KEY')


In [3]:
def gpt3 (text, engine):
    openai.api_key = OPENAI_KEY

    response = openai.Completion.create(
      engine=engine,
      prompt=  text + "Tl;dr",
      temperature=0.7,
      max_tokens=64,
      top_p=1,
      frequency_penalty=0,
      presence_penalty=0
    )
    output = response.choices[0].text

    return output

In [42]:

post =  [
        "A video clip claims to show Amber Heard “sniffing cocaine” from a tissue during her testimony in the defamation trial brought by Johnny Depp in the spring of 2022. The trial pertains to an article Heard wrote alleging that Depp abused her during their marriage. There is no evidence that Heard sniffed cocaine in the video. In fact, the longer video from which the short clip was taken shows otherwise. The claim that Heard sniffed cocaine is false.",
        "Since the high profile trial began in April 2022, fans of Depp have engaged in online trolling of Heard, including spreading rumors surrounding her conduct and statements in court. While fans of both parties have displayed such behavior, the online campaign against Heard seems particularly heated. ",
        "Tweets and TikTok posts claimed to show Heard sniffing cocaine as she paused, held a tissue to her nose, and rubbed her nose afterward.",
        "Yo I've done some shit in my day but I've never had the stones to do cocaine in a courtroom pic.twitter.com/T7xw1pLLRN",
        "how did they miss this lmao",
        "The tweet and TikTok video miscaptioned the clip, which was pulled from the second day of Heard’s testimony against Depp. The full video is available below:",
        "The clip claiming that she is doing cocaine was pulled from the 14:38 mark in the above video. ",
        "At the beginning of the video, a box of tissues is visible on the right side of the stand (Heard’s left) where Heard is making her testimony, within reaching distance. At numerous times in her testimony, Heard was visibly emotional and sniffling while speaking. At any point in the testimony she could have reached out to grab a tissue. Twelve minutes into the longer version of the video, before the alleged cocaine sniffing occurs, she is seen reaching for a tissue on the left side of the stand (her right), and wiping her nose. She then folds the tissue and puts her hands down. She brings the tissue back up a few minutes later to wipe her nose. ",
        "The TikTok video similarly misrepresented and altered the sequence of events as they took place in the trial. At around 13:39 in the actual video, Heard talks about taking Depp’s daughter to the restaurant Benihana. At 14:38 Heard presses the tissue to her nose. The TikTok video claims that the moment where she pressed the tissue to her nose came before she described the dinner, and detailed how her “lethargic” style of talking later was supposedly a “comedown” from the cocaine hit.  ",
        "There is no evidence that the tissue carried cocaine or that she was sniffing drugs during her testimony. Moreover, it strains credulity to suppose that a box of tissues provided to all witnesses by the court would be laced with cocaine, or that the defendant would brazenly snort drugs mere feet from the judge. This is yet another example of one faction attempting to manufacture evidence that confirms preconceived beliefs they apparently hold about the character of the other party in the trial. We have covered similar claims before. ",
        "Given that the tweet and TikTok both misrepresent a video that simply shows Heard sniffing and wiping her nose with a court-provided tissue, and the larger video from the trial contains no evidence of drugs being used, we rate this claim as “Miscaptioned.”"
    ]

text= ' '.join(post)
engine = "text-davinci-002"
output = gpt3 (text, engine)

In [43]:
output

': The video claiming to show Amber Heard “sniffing cocaine” during her testimony is actually a clip of her wiping her nose with a tissue. There is no evidence that Heard did anything illegal during her testimony.'

In [40]:
output

'\n\nA video clip was released that appears to show Amber Heard sniffing cocaine from a tissue during her testimony in the defamation trial against Johnny Depp. The clip was taken out of context from a longer video, and there is no evidence that Heard was actually sniffing cocaine. Despite this, fans of Depp have'

In [4]:
import hashtags_generetor
import emojis
import verify_post

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/mariabarbosa/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [5]:
def generate_capslock_allegation(data):
    
    text = re.sub(r'[^\w\s]', '', data) 
    
    
    ## Definir o modelo
    kw_model = KeyBERT()
    words_bert = [word for word, value in kw_model.extract_keywords(docs=text, keyphrase_ngram_range=(1,1))]
                                        
    allegation =  []
    
    for word in text.strip('\n').split():
        if word.lower() in words_bert:
            allegation.append(word.upper())
        else:
            allegation.append(word)
    
    allegation = ' '.join(allegation)
    post = []

    #emoji.emojize(':thumbs_up:')
    post.append(allegation)
    #+ choice(dic[data['evaluation']])
    return post

In [6]:
def interjections (evaluation):
    with open('../corpus/interjections.json') as f:
        interjection = json.load(f)
        if evaluation == "true":
            l = random.choice (interjection["true"])
        else:
            l = random.choice (interjection["fake"])
        return l
 

In [15]:
def main():
    posts = {}
    ##### Read the json files with de news
    for file in os.listdir("../../extractors/Snopes/exemplo"):
        with open("../../extractors/Snopes/exemplo/" + file) as f:
            data = json.load(f)
            f.close()
  
        text= ' '.join(data['postText'])
        engine = "text-curie-001" #text-davinci-002"
        output = gpt3 (text, engine)

        post = generate_capslock_allegation(output)
        if (verify_post.verify_length(post) == False):
          post = post[:280]



       
        hashtags = hashtags_generetor.generateHashtags(' '.join(data['postText']).split('\n'))
        

        hashtags = [' #'+h for h in hashtags]
        hashtags.append ('#'+data['evaluation'])
        post.append ( ' '.join (hashtags))
        
       # post.append (emojis.emojis_selector(data['evaluation']))
        
       # post.append (interjections (data['evaluation']))
        
        seed = random.randint(0,10)
        if verify_post.verify_length(' '.join (post)):
          post = post[:280]

        
        while (verify_post.verify_length(' '.join (post)) and seed > 0):
                seed = seed - 1
                post.append(emojis.emojis_selector(data['evaluation']))
        

        random.shuffle(post) 
        post.append(data['url'])

        post = ' '.join(post)
        print(post)


        df = pd.read_csv('snopes_posts_gtp3_v1.csv')

        row = {'id': data['id'], 'post': post}

        # Adicionar a nova linha ao dataset
        df = df.append(row,  ignore_index=True)

        #Retirar colunaas que estão a mais
        df = df[['id','post' ]]


        df.to_csv('snopes_posts_gtp3_v1.csv')
            
        print('--------------------------------------------')
        f.close()
        
        

        
        

In [16]:
main()

This is not a GENUINE TWEET from TRUMP  #account  #rule  #rumor  #risk #false https://www.snopes.com/fact-check/musk-reinstate-trump-on-twitter/
--------------------------------------------
😴 A Reddit post claiming that NASAS MARS Curiosity ROVER captured an IMAGE of an alleged portal and wall nearby that looks artificial was as incredulous as it seemed because what was depicted in the grainy blackandwhite version of the PHOTOGRAPH was actually a 🔫  #curiosity #false https://www.snopes.com/fact-check/nasa-mars-portal-door/
--------------------------------------------
😪 #correct-attribution 😔 Karine JEANPIERRE said that FOX NEWS is RACIST before the CORONAVIRUS and they will be RACIST after the CORONAVIRUS 🔫 😒 https://www.snopes.com/fact-check/jean-pierre-fox-news-racist/
--------------------------------------------
😪  #cocaine  #nose  #testimony #miscaptioned 😡 There is no EVIDENCE that AMBER Heard SNIFFED COCAINE in the Trial Pertaining to JOHNNY Depp https://www.snopes.com/fact-check

In [14]:
main()

This is a fake TWEET that was created as part of a social media fantasy involving Elon MUSK taking over TWITTER and reinstating DONALD Trumps account The REALDONALDTRUMP account is still suspended and MUSK has not been successful in his bid to takeover TWITTER  #account  #trump #false https://www.snopes.com/fact-check/musk-reinstate-trump-on-twitter/
--------------------------------------------
No 💀 😭 😪 😡 😩 #false 😭 😫 💀 https://www.snopes.com/fact-check/nasa-mars-portal-door/
--------------------------------------------
😡 Yes KARINE JEANPIERRE called FOX News RACIST in 2020 and the remark is still available to view on her Twitter account 💀 🔫 😫 😭 😪 #correct-attribution https://www.snopes.com/fact-check/jean-pierre-fox-news-racist/
--------------------------------------------
A video clip claims to show AMBER Heard SNIFFING COCAINE from a tissue during her testimony in the defamation trial brought by Johnny Depp in the spring of 2022 The trial pertains to an article Heard wrote alleging 